# Item-Item Top-N Recommendations

In this excercise we will implement a simple top-N recommender, evaluate the algorithms, and then call algorithms from the Surprise package. In top-N recommendations the algorithm is requested to produce a list of n items that the user will be interested in. 
In this particular execercise we will work with the escape room dataset that we have discussed in class.

First, let's load the dataset, which is already split by time into a training set and a test set:

In [1]:
import pandas as pd
from collections import defaultdict

train_set_path = 'resources//train_numerized_with_anon.csv'
test_set_path = 'resources//test_numerized_with_anon.csv'

train_set = pd.read_csv(train_set_path, parse_dates=[3], index_col='index')
test_set = pd.read_csv(test_set_path, parse_dates=[3], index_col='index')

users_in_train = train_set.userID.unique()

# Filter out new users from the test set
test_set = test_set[test_set.userID.isin(users_in_train)]

In [2]:
print('Train set size: ' + str(train_set.shape))
print('Test set size: ' + str(test_set.shape))

Train set size: (40022, 4)
Test set size: (687, 4)


In [3]:
# Let's take a look at the 5 first rows on the training set
train_set.head()

,userID,itemID,rating,timestamp
index,,,,
0,0,0,10,11/11/2015
1,1,0,10,11/11/2015
2,2,0,8,11/11/2015
3,3,0,10,11/11/2015
4,4,0,10,11/11/2015


## Part 1: Recommend Most Popular Items 

Now we can begin implementing the our first algorithm, that recommends to the user the list of most popular items. Although this is not a personalized approach, in many cases, this is not a bad idea - popular items are popular because everybody choose them, so there is a high likelihood that recommended popular items will be indeed chosen by the user.

In the code below, fill in the missing parts. The algorithm has a training method, where item popularity is computed, and a recommendation method, where the list of popular items.

In [4]:
class MostPopular:

    def __init__(self):
        self.item_ratings_sorted = None
        self.train_set = None

    def learn_model(self, train_set):
        self.train_set = train_set
        self.item_ratings_sorted = train_set['itemID'].value_counts()

    def get_top_n_recommendations(self, test_set, top_n):
        result = {}
        already_ranked_items_by_users = self.train_set.groupby('userID')['itemID'].apply(list)
        top_items_list = self.item_ratings_sorted.keys().tolist()
        for userID in test_set.userID.unique():
            result[str(userID)] = [i for i in top_items_list if i not in already_ranked_items_by_users[userID]][:top_n]
        return result

    def clone(self):
        pass


Now we can call the most popular algorithm to deliver a list of reocmmendations. The code below prints the list of top 5 recommended items for user with ID 431.

In [5]:
popular = MostPopular()
popular.learn_model(train_set)
popular_recs = popular.get_top_n_recommendations(test_set,top_n=5)
print(popular_recs['431'])
assert popular_recs['431'] == [53, 26, 68, 85, 16], 'Wrong set of popular items returned..'

[53, 26, 68, 85, 16]


## Part 2 - Item-Item Recommendations

We now learn a slightly more sophisticated model, that uses item-item similarities. Given such a similarity score, we can recommend to a user items that are most similar to the items that the user has chosen in the past. One such useful similarity metric is the Jaccard coefficient. For two items i1 and i2, the Jaccard similarity is the number of users who have chosen both i1 and i2, divided by the number of users who have chosen either i1 or i2. That is, given the list of users who have chosen i1 and the list of users who have chosen i2, the Jaccard similarity is the intersection of the lists, divided by the union of the lists.

In practice, to expedite the recommendation process, and hence reduce online latency, we will compute the item-item co-occurence matrix in the model learning phase. Then, online, when recommendations are requested, we only need to compute for each item that the user has already chosen in the past, the Jaccard scores for the other items.

As the user has chosen several items in the past, we need to aggregate the Jaccard scores. That is, if the user has previously chosen i1 and i2, item i3 has two scores J(i1,i3) and J(i2,i3), and an aggregation of the scores is needed. There are two popular aggregation functions - sum and max. Empirically, max typically has better perfromance.

Fill in the missing parts in the code below.

In [6]:
from tqdm import tqdm
import numpy as np
import operator

class Jaccard:

    def __init__(self):
        self.item_ratings_sorted = None
        self.train_set = None
        self.item_item_counts = {}
        self.item_counts = None
        
    def learn_model(self, train_set):
        print('Started training')
        self.train_set = train_set
        self.item_counts = self.train_set.groupby('itemID')['userID'].agg('count')

        users_grouped_items = train_set.groupby('userID')
        for user, user_data in tqdm(users_grouped_items):
            if len(user_data.index) < 3:
                continue

            picked_items = user_data.itemID.tolist()

            for i1 in range(len(picked_items)):
                item1 = picked_items[i1]

                for i2 in range(i1 + 1, len(picked_items) - 1):
                    item2 = picked_items[i2]

                    if item1 not in self.item_item_counts:
                        self.item_item_counts[item1] = {}
                    if item2 not in self.item_item_counts[item1]:
                        self.item_item_counts[item1][item2] = 0
                    self.item_item_counts[item1][item2] += 1
                    if item2 not in self.item_item_counts:
                        self.item_item_counts[item2] = {}
                    if item1 not in self.item_item_counts[item2]:
                        self.item_item_counts[item2][item1] = 0
                    self.item_item_counts[item2][item1] += 1
        print('Done training')

    def get_top_n_recommendations(self, test_set, top_n):
        print('Started computing recommendations')
        result = {}
        already_ranked_items_by_users = self.train_set.groupby('userID')['itemID'].apply(list)

        for userID in tqdm(test_set.userID.unique().tolist()):
            result[str(userID)] = []
            maxvalues = defaultdict(int)

            for i in already_ranked_items_by_users[userID]:
                if i not in self.item_item_counts:
                    continue
                    
                items = self.item_item_counts[i]

                for j in items:
                    if j in already_ranked_items_by_users[userID]:
                        continue

                    if items[j] > 10:
                        d = items[j] / (self.item_counts[i] + self.item_counts[j] - items[j])
                        if d > maxvalues[j]:
                            maxvalues[j] = d

            top_list = sorted(maxvalues.items(), key=lambda kv: -kv[1])
            result[str(userID)] = [x[0] for x in top_list[:top_n]]
        print('Done computing recommendations')
        return result

    def clone(self):
        pass


The code below trains a Jaccard model and generates recommendations. Training will take a while, as we need to iterate over all users, and for each user go over her items in quadratic time.

In [7]:
jaccard = Jaccard()
jaccard.learn_model(train_set)
jaccard_recs = jaccard.get_top_n_recommendations(test_set,top_n=5)


Started training


100%|██████████████████████████████████████████████████████████████████████████| 20197/20197 [00:02<00:00, 9654.47it/s]


Done training
Started computing recommendations


100%|████████████████████████████████████████████████████████████████████████████████| 227/227 [00:24<00:00, 17.28it/s]


Done computing recommendations


## Part 3 - Comparing the Algorithms 

We now want to compare the recommendation lists to see which one is better. In top-N recommendations it is popular to computer the Precision@N metric - the portion of recommended items that were chosen by users in the test set. This is typically a reasonable metric for real systems, where one wants to optimize the number of recommended items that are chosen.

We compute Precision@N by comparing the number of recommendations chosen by the users, divided by the number of overall recommendations.

Fill in the missing parts in the code below:

In [8]:
def compute_precision(test_set, recommendations):
    hits = 0
    recs = 0
    used = 0
    
    for u in test_set.userID.unique():
        userData = test_set[test_set.userID == u]
        used += len(userData)
        userRecs = recommendations.get(str(u))
        recs += len(userRecs)
        
        for i in userRecs:
            if i in userData['itemID'].tolist():
                hits += 1
    return hits / recs
        
    

In [9]:
p1 = compute_precision(test_set,jaccard_recs)
p2 = compute_precision(test_set,popular_recs)
print("Jaccard=", p1, "  Popularity=", p2)

Jaccard= 0.03171806167400881   Popularity= 0.027312775330396475


The precision values for this dataset may seem pretty low, but this is typical for many top-N problems. It is important not to compute metrics that hide the true values, such as AUC, but to acknowledge the perfromance of the system in the application.

## Part 4 - Calling Algorithms from the Surprise Package

There are many existing recommendation algorithms available. We will now see how we can call algorithms from the Surprise package. To run the code below you first have to install surprise (pip install surprise).

Surprise documentation: https://surprise.readthedocs.io/en/stable/

The code below adds a wrapper around the algorithm to transform the resulting recommendations into our desired format.

In [10]:
import sys, string, os
import pandas as pd
import itertools
from tqdm import tqdm
import numpy as np
import operator
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import PredefinedKFold
from surprise.prediction_algorithms import *



class SurpriseRecMethod():

    def __init__(self, method):
        self.method = method

    def fit(self, train_set):
        self.train_set = train_set


    def get_top_n_recommendations(self, test_set, top_n):
        self.test_set = test_set

        test_path_tmp = "resources//test_file.csv"
        train_path_tmp = "resources//train_file.csv"

        self.train_set.to_csv(train_path_tmp, index=False, header=False)
        self.test_set.to_csv(test_path_tmp, index=False, header=False)

        fold_files = [(train_path_tmp, test_path_tmp)]
        reader = Reader(rating_scale=(1, 10), line_format='user item rating', sep=',')
        data = Dataset.load_from_folds(fold_files, reader=reader)

        for trainset, testset in PredefinedKFold().split(data):
            self.method.fit(trainset)

        already_ranked_items_by_users = self.train_set.groupby('userID')['itemID'].apply(list)

        recommendations = {}
        pbar = tqdm(total=len(self.test_set.userID.unique()))
        for userID in self.test_set.userID.unique():
            pbar.update(1)

            if userID not in self.train_set.userID.unique():
                recommendations[str(userID)] = []
                continue

            items_expected_ranking = {}
            for itemID in self.train_set.itemID.unique():
                if itemID in already_ranked_items_by_users[userID]:
                    continue
                # Calc prediction for item for user
                predicted = self.method.predict(str(userID), str(itemID), clip=False)
                items_expected_ranking[itemID] = predicted[3]
            sorted_predictions = sorted(items_expected_ranking.items(), key=operator.itemgetter(1))
            sorted_predictions.reverse()
            sorted_predictions = [x[0] for x in sorted_predictions]
            user_recommendations = sorted_predictions[:top_n]
            recommendations[str(userID)] = user_recommendations
        pbar.close()
        return recommendations


The code below calls the package with the SlopeOne algorithm.

In [11]:
modelSlopeOne = SurpriseRecMethod(SlopeOne())
modelSlopeOne.fit(train_set)
recSlopeOne = modelSlopeOne.get_top_n_recommendations(test_set, 5)
p3 = compute_precision(test_set,recSlopeOne)

100%|████████████████████████████████████████████████████████████████████████████████| 227/227 [00:02<00:00, 94.90it/s]


The code below calls the package with a nearest neighbor user-item recommendation method.

In [12]:
modelKNNUser = SurpriseRecMethod(KNNBasic(sim_options={'name': 'cosine', 'user_based': True}))
modelKNNUser.fit(train_set)
recKNNUser = modelKNNUser.get_top_n_recommendations(test_set, 5)
p4 = compute_precision(test_set,recKNNUser)

Computing the cosine similarity matrix...
Done computing similarity matrix.


100%|████████████████████████████████████████████████████████████████████████████████| 227/227 [00:08<00:00, 28.16it/s]


Finally, let us look at the results of all algorithms together:

In [13]:
pd.DataFrame.from_dict({'Jaccard':p1,'Popularity':p2,'SlopeOne':p3,'User KNN':p4}, orient='index',columns=['Precision'])

,Precision
Jaccard,0.031718
Popularity,0.027313
SlopeOne,0.032599
User KNN,0.075771


For this particular dataset, the user nearest neighbor approach worked best. Hence, should we need to choose a method to put online, we should go with this method.